In [1]:
import ipywidgets
import traitlets
import yt

/data/data/com.termux/files/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ds = yt.load("/home/data/IsolatedGalaxy/galaxy0030/galaxy0030")

yt : [INFO     ] 2018-03-18 09:27:40,137 Parameters: current_time              = 0.0060000200028297998
yt : [INFO     ] 2018-03-18 09:27:40,141 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2018-03-18 09:27:40,143 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-03-18 09:27:40,146 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2018-03-18 09:27:40,149 Parameters: cosmological_simulation   = 0.0


In [3]:
def display_arr(arr):
    unit_registry = arr.units.registry
    dimension = arr.units.dimensions
    equiv = [k for k, v in unit_registry.lut.items() if v[1] is dimension]
    dropdown = ipywidgets.Dropdown(options = sorted(equiv), value = str(arr.units))
    def arr_updater(arr, texts):
        def _value_updater(change):
            arr2 = arr.in_units(change['new'])
            if arr2.shape == ():
                arr2 = [arr2]
            for v, t in zip(arr2, texts):
                t.value = str(v.value)
        return _value_updater
    if arr.shape == ():
        arr_iter = [arr]
    else:
        arr_iter = arr
    texts = [ipywidgets.Text(value = str(_.value), disabled = True) for _ in arr_iter]
    dropdown.observe(arr_updater(arr, texts), names="value")
    return ipywidgets.HBox(texts + [dropdown])

In [4]:
display_arr(ds.domain_width)

HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length')))

In [5]:
def display_ds(ds):
    attrs = ["domain_width", "domain_left_edge", "domain_right_edge", "current_time"]
    lb = ipywidgets.VBox([ ipywidgets.Label(value = _) for _ in attrs])
    vb = ipywidgets.VBox([ display_arr(getattr(ds, _)) for _ in attrs])
    hb = ipywidgets.HBox([lb, vb])
    return hb

In [6]:
display_ds(ds)

HBox(children=(VBox(children=(Label(value='domain_width'), Label(value='domain_left_edge'), Label(value='domain_right_edge'), Label(value='current_time'))), VBox(children=(HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Text(value='0.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Text(value='1.0', disabled=True), Dropdown(index=9, options=('AU', 'Angstrom', 'R_earth', 'R_jup', 'R_sun', 'Rsun', 'angstrom', 'au', 'cm', 'code_length', 'ft', 'l_geom', 'l_pl', 'ly', 'm', 'mile', 'pc', 'r_earth', 'r_jup', 'r_sun', 'rsun', 'solRad', 'unitary'), value='code_length'))), HBox(children=(Text(value='0.0060000200028298', disabled=True), Dropdown(options=('code_time', 'd', 'day', 'hr', 'min', 's', 't_geom', 't_pl', 'yr'), value='code_time')))))))

In [7]:
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics

In [8]:
class VariableInspectorWindow(object):
    instance = None

    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")

        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell

        self._box = widgets.Box()
        self._box.layout.overflow_y = 'scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._table.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table></div>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable
        Inspector."""
        self._box._ipython_display_()


In [9]:
inspector = VariableInspectorWindow(get_ipython())
inspector

Box(children=(HTML(value='Not hooked'),), layout=Layout(overflow_y='scroll'))

In [10]:
v = 'this'

In [11]:
ds.fields

Parsing Hierarchy : 100%|██████████| 173/173 [00:00<00:00, 1234.86it/s]
yt : [INFO     ] 2018-03-18 09:27:41,548 Gathering a field list (this may take a moment.)


In [12]:
ds.fields.gas.density.sampling_type

'cell'

In [13]:
dir(ds.fields.gas.density)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_copy_def',
 '_function',
 '_ion_to_label',
 '_is_ion',
 '_unit_registry',
 'check_available',
 'dimensions',
 'display_field',
 'display_name',
 'ds',
 'get_dependencies',
 'get_label',
 'get_latex_display_name',
 'get_projected_units',
 'get_source',
 'get_units',
 'name',
 'nodal_flag',
 'not_in_all',
 'output_units',
 'particle_type',
 'sampling_type',
 'take_log',
 'unit_registry',
 'units',
 'validators',
 'vector_field']

In [14]:
density = ds.fields.gas.density

In [32]:
import inspect

In [35]:
inspect.getmembers(density._function)

[('__annotations__', {}),
 ('__call__',
  <method-wrapper '__call__' of function object at 0x7a188ea62158>),
 ('__class__', function),
 ('__closure__', (<cell at 0x7a188ed3e5b8: tuple object at 0x7a188ea61808>,)),
 ('__code__',
  <code object _TranslationFunc at 0x7a189a4f7c00, file "/data/data/com.termux/files/home/yt/yt/yt/fields/derived_field.py", line 40>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x7a188ea62158>),
 ('__dict__', {'alias_name': ('enzo', 'Density')}),
 ('__dir__', <function function.__dir__>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x7a188ea62158>),
 ('__format__', <function function.__format__>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x7a188ea62158>),
 ('__get__', <method-wrapper '__get__' of function object at 0x7a188ea62158>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x7a188ea62158>),
 ('__globals__',
  {'Deriv

In [38]:
inspect.getframeinfo(density._function.__closure__[0].frame)

AttributeError: 'cell' object has no attribute 'frame'

In [43]:
inspect.getclosurevars(density._function)

ClosureVars(nonlocals={'field_name': ('enzo', 'Density')}, globals={}, builtins={}, unbound={'copy'})

In [56]:
def display_field_source(fobj):
    source = ipywidgets.Textarea(value = fobj.get_source())
    ac = ipywidgets.Accordion([source])
    ac.set_title(0, "Source")
    return ac

In [58]:
ac = display_field_source(density)

In [59]:
ac.traits()

{'_display_callbacks': <traitlets.traitlets.Instance at 0x7a189d30d470>,
 '_dom_classes': <traitlets.traitlets.Tuple at 0x7a189d308518>,
 '_model_module': <traitlets.traitlets.Unicode at 0x7a189d2c0be0>,
 '_model_module_version': <traitlets.traitlets.Unicode at 0x7a189d2c0c88>,
 '_model_name': <traitlets.traitlets.Unicode at 0x7a189d21d3c8>,
 '_msg_callbacks': <traitlets.traitlets.Instance at 0x7a189d30d198>,
 '_property_lock': <traitlets.traitlets.Dict at 0x7a189d30db00>,
 '_states_to_send': <traitlets.traitlets.Set at 0x7a189d30d7b8>,
 '_titles': <traitlets.traitlets.Dict at 0x7a189d21d2b0>,
 '_view_count': <traitlets.traitlets.Int at 0x7a189d30dcf8>,
 '_view_module': <traitlets.traitlets.Unicode at 0x7a189d2c0cc0>,
 '_view_module_version': <traitlets.traitlets.Unicode at 0x7a189d2c0cf8>,
 '_view_name': <traitlets.traitlets.Unicode at 0x7a189d21d390>,
 'box_style': <traitlets.traitlets.CaselessStrEnum at 0x7a189d24dfd0>,
 'children': <traitlets.traitlets.Tuple at 0x7a189d24de48>,
 'c

In [60]:
ac.children

(Textarea(value='    def _TranslationFunc(field, data):\n        # We do a bunch of in-place modifications, so we will copy this.\n        return data[field_name].copy()\n'),)